In [2]:
import collections
import functools
# Read the file and open the capture by creating an OrbitCapture instance.
# from orbitutils.orbit_capture import OrbitCapture
import pandas as pd

callstack_csv = pd.read_csv("~/cumbia_callstack.csv").to_numpy()
frametrack_csv = pd.read_csv("~/cumbia_frametrack.csv").to_numpy()

callstack_csv = callstack_csv[callstack_csv[:, 1].argsort()]

In [3]:
import numpy as np

def callstacks(callstack_csv, frametrack_csv):
  frame_index_to_callstacks = {}
  frame_index = 0
  frame_ends = frametrack_csv[:, 3]
  frame_end =frame_ends[frame_index]
  frame_index_to_callstacks[frame_index] = []
  callstack_names = [s.split('/') for s in  callstack_csv[:, 2]]
  timestamps = callstack_csv[:, 1]
  for i in range(callstack_csv.shape[0]):
    timestamp = timestamps[i]

    while timestamp > frame_end:
      if frame_index + 1 == frametrack_csv.shape[0]: 
        return frame_index_to_callstacks
      frame_index += 1
      frame_end = frame_ends[frame_index]
      frame_index_to_callstacks[frame_index] = [] 
    frame_index_to_callstacks[frame_index].append(callstack_names[i])
  return frame_index_to_callstacks

In [4]:
frame_index_to_callstacks = callstacks(callstack_csv, frametrack_csv)

In [5]:
frame_durations = frametrack_csv[:, 4]

In [15]:
def get_exclusive_count(function_name, callstacks):
  exclusive_count = 0;
  for callstack in callstacks:
      if (len(callstack) > 0 and callstack[0] == function_name):
        exclusive_count += 1
  return exclusive_count

In [58]:
def time_per_frame(frame_index_to_callstack, frame_durations):
  name_to_hits = {}
  for callstacks in frame_index_to_callstacks.values():
    for callstack in callstacks:
      head = callstack[0]
      name_to_hits.setdefault(head, 0)
      name_to_hits[head] += 1
  total_callstacks = sum(len(s) for s in frame_index_to_callstacks.values())
  mean_frame_time = frame_durations.mean()
  return dict([(name, (mean_frame_time *hits/total_callstacks/1e6)) for name, hits in name_to_hits.items()])

In [59]:
name2time = time_per_frame(frame_index_to_callstacks, frame_durations)

In [30]:
def time_per_frame2(function_name, frame_index_to_callstack, frame_durations):
  total_time = 0
  for frame_index, callstacks in frame_index_to_callstack.items():
    rate = get_exclusive_count(function_name, callstacks) / len(callstacks)
    total_time += frame_durations[frame_index] * rate
  return total_time/1e6/frame_durations.size

In [38]:
def get_all_names(frame_index_to_callstack):
  result = []
  for callstacks in frame_index_to_callstacks.values():
    for callstack in callstacks:
      result += callstack
  return np.unique(result)

In [36]:
names = get_all_names(frame_index_to_callstacks)

In [47]:
def compare_time_per_frame(function_name, frame_index_to_callstacks, frame_durations):
  t1 = time_per_frame(function_name, frame_index_to_callstacks, frame_durations)
  t2 = time_per_frame2(function_name, frame_index_to_callstacks, frame_durations)
  print(abs(t1-t2) / t2)
  return abs(t1-t2) / t2

In [48]:
max(map(lambda x : compare_time_per_frame(x,frame_index_to_callstacks, frame_durations ), names))

0.011926312004410135
0.001351467089160045
0.011260006658262331
0.0028185071512641826
0.005321967074776933
0.01791814892155093
0.02475608889309781
0.010222595911278363


/tmp/ipykernel_1983293/2054096210.py:4: RuntimeWarning: invalid value encountered in double_scalars
  print(abs(t1-t2) / t2)
/tmp/ipykernel_1983293/2054096210.py:5: RuntimeWarning: invalid value encountered in double_scalars
  return abs(t1-t2) / t2


nan
0.026099173824547985
0.006723154666484884
0.021482533902768962
0.0021636518691299867
0.015046811131485635
nan
0.001743126608292873
0.009698545480839062
0.002285004372601309
0.006696507182651216
0.0005536716120434846
0.014805820047624447
0.04483663642916222
0.0020701963961517545
0.004382374799078062
0.013960914416225326
0.001533360306751266
0.04266880406079179
0.01047232153979111
0.003010983355350477
0.0054716980446879
nan
0.012483680989999857
0.0013751939511647846
0.03390748179050543
0.015016825304434623
0.04499235221301583
0.022909934191231513
0.0015255114339190114
0.041109049273096256
0.03863121668968187
0.00014449210448605146
0.0967988696916765
nan
0.0122195703344928
0.03140276510112005
0.01007113869756157
0.0030859775101162554
0.030571002417397446
0.062327656920985366
nan
0.016676739694375557
0.0021216868791627324
0.0065180881644860715
0.006113372124426773
0.004233556724149805
0.0049796529127078765
0.044410734304753514
0.055506279976484864
0.014084610418473972
0.022336935514830

KeyboardInterrupt: 

In [46]:
len(names)

2705

In [60]:
name2time["eva::math::dot(eva::math::vector<float, 4ul> const&, eva::math::vector<float, 4ul> const&)"]

0.8460718705759032